<a href="https://colab.research.google.com/github/tqsyed/confNorm/blob/main/CE_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100%|██████████| 170498071/170498071 [00:05<00:00, 29491182.42it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [ ]:
import math
import torch.optim as optim

f = nn.Sigmoid()

def ratio_loss(output, target):
       #if (sum(output) - output[target]) == 0:
         #print((sum(output) - output[target]))
         #return 0
       return output[target] / (sum(output))# - output[target])
class new_loss(nn.CrossEntropyLoss):
  def forward(self, input, target):
        val = 0
        for i in range(0,len(input)):
          val +=  ratio_loss(f(input[i]), target[i])
        # print("Cross Entropy Value")
        # print(F.cross_entropy(input, target, weight=self.weight, ignore_index=self.ignore_index, reduction=self.reduction))
        # print("Ratio Value")
        # print(val)
        if(math.isnan(val)):
          print(val)
          return F.cross_entropy(input, target, weight=self.weight, ignore_index=self.ignore_index, reduction=self.reduction)
        #print(F.cross_entropy(input, target, weight=self.weight, ignore_index=self.ignore_index, reduction=self.reduction))
        return F.cross_entropy(input, target, weight=self.weight, ignore_index=self.ignore_index, reduction=self.reduction) # - (0.01 *  val)
        #return val



criterion = new_loss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)

for epoch in range(20):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.166
[1,  4000] loss: 1.887
[1,  6000] loss: 1.693
[1,  8000] loss: 1.619
[1, 10000] loss: 1.524
[1, 12000] loss: 1.476
[2,  2000] loss: 1.403
[2,  4000] loss: 1.394
[2,  6000] loss: 1.345
[2,  8000] loss: 1.343
[2, 10000] loss: 1.327
[2, 12000] loss: 1.313
[3,  2000] loss: 1.230
[3,  4000] loss: 1.249
[3,  6000] loss: 1.226
[3,  8000] loss: 1.226
[3, 10000] loss: 1.234
[3, 12000] loss: 1.210
[4,  2000] loss: 1.142
[4,  4000] loss: 1.149
[4,  6000] loss: 1.154
[4,  8000] loss: 1.137
[4, 10000] loss: 1.124
[4, 12000] loss: 1.137
[5,  2000] loss: 1.075
[5,  4000] loss: 1.045
[5,  6000] loss: 1.052
[5,  8000] loss: 1.085
[5, 10000] loss: 1.086
[5, 12000] loss: 1.091
[6,  2000] loss: 0.973
[6,  4000] loss: 1.022
[6,  6000] loss: 1.028
[6,  8000] loss: 1.040
[6, 10000] loss: 1.022
[6, 12000] loss: 1.027
[7,  2000] loss: 0.931
[7,  4000] loss: 0.965
[7,  6000] loss: 0.980
[7,  8000] loss: 0.971
[7, 10000] loss: 0.995
[7, 12000] loss: 1.007
[8,  2000] loss: 0.901
[8,  4000] 

In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

AttributeError: ignored

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

In [ ]:

confidences = []
for i in range(0,10):
  confidences.append([])
confidences

class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))




with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1
            confidences[labels[i]].append(f(outputs[i]))


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))




In [ ]:
from sklearn.metrics import confusion_matrix

nb_classes = 10

# Initialize the prediction and label lists(tensors)
predlist=torch.zeros(0,dtype=torch.long, device='cpu')
lbllist=torch.zeros(0,dtype=torch.long, device='cpu')


with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        # for i in range(4):
        #     label = labels[i]
        #     pred = c[i]

        predlist=torch.cat([predlist,predicted.view(-1).cpu()])
        lbllist=torch.cat([lbllist,labels.view(-1).cpu()])



# with torch.no_grad():
#     for i, (inputs, classes) in enumerate(dataloaders['val']):
#         inputs = inputs.to(device)
#         classes = classes.to(device)
#         outputs = model_ft(inputs)
#         _, preds = torch.max(outputs, 1)

#         # Append batch prediction results
#         predlist=torch.cat([predlist,preds.view(-1).cpu()])
#         lbllist=torch.cat([lbllist,classes.view(-1).cpu()])

# Confusion matrix
conf_mat=confusion_matrix(lbllist.numpy(), predlist.numpy())
print(conf_mat)

# Per-class accuracy
class_accuracy=100*conf_mat.diagonal()/conf_mat.sum(1)
print(class_accuracy)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(lbllist.numpy(), predlist.numpy()))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def hist(cno):
  df2 = pd.DataFrame()
  for i in range(0,len(confidences[cno])):
    df2 = df2.append(pd.DataFrame(confidences[cno][i].numpy().reshape(-1, len(confidences[cno][i].numpy()))))


  corr = df2.corr()
  corr.style.background_gradient(cmap='coolwarm')

  arr = []
  for i in range(0,10):
    arr.append(sum(df2.iloc[:,i])/1000)

  plt.bar(np.arange(len(arr)),arr)
  plt.show()

In [ ]:
for i in range(0,10):
  print(i)
  hist(i)

In [ ]:
class correct


In [ ]:
import pandas as pd
df2 = pd.DataFrame()


for i in range(0,len(outputs)):
  df2 = df2.append(pd.DataFrame(outputs[i]))

import matplotlib.pyplot as plt

plt.matshow(df2.corr())
plt.colorbar()
plt.show()

In [ ]:
import collections
mc = []
for i in range(0,10):
  mc.append([])


df2 = pd.DataFrame()
for nu in range(0, 10):
  results = []
  with torch.no_grad():
      for data in testloader:
          images, labels = data
          outputs = net(images)
          _, predicted = torch.max(outputs, 1)
          c = (predicted == labels).squeeze()
          for i in range(4):
              label = labels[i]
              class_correct[label] += c[i].item()
              class_total[label] += 1
              if(label == nu):
                results.append(predicted[i].item())
              #confidences[labels[i]].append(f(outputs[i]))

  print(results)
  mc[nu] = results
  #df2 = df2.append(pd.DataFrame(np.array(list(collections.Counter(results).values())).reshape(-1, len(np.array(list(collections.Counter(results).values()))))))




In [ ]:
df2 = pd.DataFrame()
for i in range(0,10):
  df2 = df2.append(pd.value_counts(np.array(mc[i])).sort_index(), ignore_index = True)



In [ ]:
import matplotlib.pyplot as plt
df2 = df2/1000
plt.matshow(df2)
plt.colorbar()
plt.show()
df2/1000